# Black Friday--Cluster

## Dataset Description

"Dataset of 550 000 observations about the black Friday in a retail store, it contains different kinds of variables either numerical or categorical. It contains missing values."

## Raw Data

In [2]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from scipy import stats
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import math

In [3]:
df_allData=pd.read_csv('BlackFriday.csv')
print(df_allData.sample(n=5))

        User_ID Product_ID Gender    Age  Occupation City_Category  \
21390   1003382  P00332342      F   0-17          10             A   
147892  1004808  P00117542      M  36-45           0             A   
172461  1002676  P00026042      M    55+          20             B   
389880  1006001  P00215742      F  26-35           7             A   
533742  1004160  P00114242      M  36-45          20             A   

       Stay_In_Current_City_Years  Marital_Status  Product_Category_1  \
21390                           2               0                   5   
147892                          1               1                  18   
172461                          1               1                   8   
389880                          0               1                   5   
533742                         4+               1                  13   

        Product_Category_2  Product_Category_3  Purchase  
21390                  8.0                 NaN      7002  
147892                

raw data has 12 columns and there are a lot of missing values in "Product_Category_2" and "Product_Category_3". Which means some of the product whould just have one category.

## Data Pre-processing

In our opinion, we think our model would do cluster on different people, so our key is people.

But the dataset includes different records of one-single people. So at the beginning we use gruopby to get each people's whole records. 

Then we try to get the mode of each one's "Product_Category_1" to represent the main product category and get the mean of one people's whole "Purchase" as a feature of "average purchase". (We are not sure weather this is a good way buy we have to do this because we cannot keep all the data to train)

What's more, we change the "Gender" attribute to 0-1 attribute.

In [4]:
groupByUserData=df_allData.groupby(['User_ID'])

times=df_allData['User_ID'].value_counts()
times=times.sort_index()

#get the mean
meanData=groupByUserData.mean()

#get the mode
modeData=groupByUserData.agg(lambda x: stats.mode(x)[0][0])

mean_mode_data={'Gender':modeData['Gender'],'Occupation':modeData['Occupation'],'Age':modeData['Age'],'City_Category':modeData['City_Category'],'Marital_Status':modeData['Marital_Status'],'Product_CateGory_1':modeData['Product_Category_1'],'Stay_In_Current_City_Years':modeData['Stay_In_Current_City_Years']}
mean_mode_data=pd.DataFrame(mean_mode_data)
mean_mode_data['times']=times
mean_mode_data['Gender_M']=pd.get_dummies(mean_mode_data['Gender'])['M']
mean_mode_data=mean_mode_data.drop(['Gender'],axis=1)
mean_mode_data['Purchase']=meanData['Purchase']

print (mean_mode_data.sample(5))

D:\Users\Changlin Jiang\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


         Occupation    Age City_Category  Marital_Status  Product_CateGory_1  \
User_ID                                                                        
1002479           1  26-35             C               0                   1   
1001835          19  26-35             B               0                   5   
1002234           1    55+             C               1                   8   
1005886          20  26-35             A               0                   1   
1003454           0  18-25             A               1                   1   

        Stay_In_Current_City_Years  times  Gender_M      Purchase  
User_ID                                                            
1002479                          2     19         0  10839.263158  
1001835                          3    435         1  10162.694253  
1002234                          0     12         1  11402.083333  
1005886                          2    255         1   9082.898039  
1003454                        

## Feature Extraction

This is the hardest part of our cluster project.

There are two key problems we have to face:

##### 1. **How to handle the discrete attributes?**
    
    There are a lot of disordered discrete attributes in our data, likes "Marital_Status", "Gender" and "Product_Category_1", we cannot just simplely calculate their euclidean distance.

##### 2. **How to evaluate our feature extraction performance?**

    Since we have so many choices of extracting the features and we do not know how to assign the weights on these feautres and this is a cluster problem, we had not a clearly mind of have to evaluate our work when we doing the feature extraction. And it is unrealistic to try all the choices and train them then evaluate the final models. The best solution is that we can find some explainable output of our feature extraction.
    
#### Deal with discrete attributes

To solve the key problems 1, we came up with ideas.

1. do one-hot encoding on the discrete attributes

1. use k-modes or k-prototype model

1. drop those discrete attributes

But their would raise some new problems if we use these solutions:

    one-hot encoding would make the features very sparse.
    
    It is not easy to combine the VDM distance and Minkowski distance together.
    
    Some discrete features may be very important like occupation but it would have a lot of possible value.
    
#### Deal with evaluation

We do not find a good way to solve this problem, we just living with it, but we still did some tries. Likes using the average purchase to evaluate the cluster output. And we use Calinski-Harabasz score and Value Difference Metrix to evalue our final clusters.

## Data Pre-processing Before Training

Since we have features of different units, we must do the data standardization and assign different weights to different features.

We choose min-max standardization and assign the weights by feeling.

## Specific Model 1: K-modes by Chaoji Zuo

I try to use the k-modes only on the categorical values to train a model, then use one-hot encoidng data to calculate the jaccard distance and euclidean distance.

In [ ]:
from kmodes.kmodes import KModes

In [5]:
X=pd.DataFrame({'Gender':modeData['Gender'],'Occupation':modeData['Occupation'],'Age':modeData['Age'],'City_Category':modeData['City_Category'],'Marital_Status':modeData['Marital_Status'],'Product_CateGory_1':modeData['Product_Category_1'],"Stay_In_Current_City_Years":modeData["Stay_In_Current_City_Years"]})

one_hot_city=pd.get_dummies(mean_mode_data['City_Category'])
one_hot_age=pd.get_dummies(mean_mode_data['Age'])
one_hot_occupation=pd.get_dummies(mean_mode_data['Occupation'])
one_hot_years=pd.get_dummies(mean_mode_data['Stay_In_Current_City_Years'])
one_hot_product=pd.get_dummies(mean_mode_data['Product_CateGory_1'])
XX=pd.concat([one_hot_age,one_hot_city,one_hot_occupation,one_hot_years,one_hot_product],axis=1)
XX['Gender_M']=mean_mode_data['Gender_M']
XX['Marital_Status']=mean_mode_data['Marital_Status']

print ("categorical data:")
print(X.sample(2))
print("one-hot encoding data:")
print(XX.sample(2))


categorical data:
        Gender  Occupation    Age City_Category  Marital_Status  \
User_ID                                                           
1005051      F           7  51-55             C               0   
1005769      M          14  46-50             B               1   

         Product_CateGory_1 Stay_In_Current_City_Years  
User_ID                                                 
1005051                   5                          0  
1005769                   1                          1  
one-hot encoding data:
         0-17  18-25  26-35  36-45  46-50  51-55  55+  A  B  C  \
User_ID                                                          
1002209     0      0      0      1      0      0    0  1  0  0   
1000982     0      0      1      0      0      0    0  0  0  1   

              ...        8  10  11  12  13  15  16  18  Gender_M  \
User_ID       ...                                                  
1002209       ...        0   0   0   0   0   0   0   0       

In [6]:
from sklearn.metrics import jaccard_similarity_score

jcArr=[]
for i in range(2,10):
    km=KModes(n_clusters=i)
    y=km.fit_predict(X)
    tempArr=[]
    for j in range(i):
        #print(sum(y==j))
        #print(XX[y==j].mode())
        jcscore=[]
        for k in XX[y==j].T:
            try:
                jcscore.append(jaccard_similarity_score(XX.loc[k],XX[y==j].mode().T[0]))
            except:
                #print(XX.loc[k].T)
                print(XX[y==j].mode())
                hhh=XX[y==j].mode()
                #print(k)
                break;
        #print(np.mean(jcscore))
        tempArr.append(np.mean(jcscore))
    print("n_cluster =",i,":",np.mean(tempArr))
    jcArr.append(np.mean(tempArr))



NameError: name 'KModes' is not defined

In [ ]:
ecArr=[]
for i in range(2,10):
    km=KModes(n_clusters=i)
    y=km.fit_predict(X)
    tempArr=[]
    for j in range(i):
        #print(sum(y==j))
        #print(XX[y==j].mode())
        ecscore=[]
        for k in XX[y==j].T:
            if(k):
                ecscore.append(np.linalg.norm(np.array(XX.loc[k])-np.array(XX[y==j].mode().T[0])))
        #print(np.mean(ecscore))
        tempArr.append(np.mean(ecscore))
    print("n_cluster =",i,":",np.mean(tempArr))


In [ ]:
np.linalg.norm(np.array(XX.loc[1004491])-np.array(XX.mode().T))

## Specific Model 2: XXX by Shuyu Chen

## Specific Model 3: ROCK Clustering by Changlin Jiang
- After searching relavant paper dealing with discrete features clustering,we decide to implement a algorithm termed ROCK.  
- The main idea is:  
    - Compute the Jaccard score between 2 data points.Input a threshold theta such that if the Jaccard score is larger than the threshold,say these 2 points are neighbors (i.e. they have a Neighbor label which is originally 0 and can be set to 1 if their Jaccard score is larger than the threshold theta.)  
    - Compute the Neighbor label of every 2 data,put them in a Neighbors Matrix A,A[i,j]=1 if point i and j are neighbors,A[i,j]=0 if not.
    - Initialization:If note the number of data points with n,initialize the clusters with n clusters,each cluster includes only one data point.
    - Gooodness:The goodness of 2 clusters $C_i$ and $C_j$ can be determined by: $$goodness=\frac{link(C_i,C_j)}{(n1+n2)^{1+2f(\theta)}-n1^{1+2f(\theta)}-n2^{1+2f(\theta)}}$$In this formula,link(Ci,Cj) is the total number of neighbors of the two clusters $C_i$ and $C_j$,$$f(\theta)=\frac{1-\theta}{1+\theta}$$is the penalty if the size of the cluster has got too many members.
    - Compute goodness of every 2 different clusters,find the pair of clusters with the largest goodness.
    - Process:Merge the two clusters into one clusters and loop the whole process untill converge or number of clusters is smaller than a wanted number.

In [7]:
X=np.array(X)
X0=list(X[:,0])
X2=list(X[:,2])
X3=list(X[:,3])
X6=list(X[:,6])

X0=[0 if x=='M' else 1 for x in X0]

X2=[0 if x=='0-17' else x for x in X2]
X2=[1 if x=='18-25' else x for x in X2]
X2=[2 if x=='26-35' else x for x in X2]
X2=[3 if x=='36-45' else x for x in X2]
X2=[4 if x=='46-50' else x for x in X2]
X2=[5 if x=='51-55' else x for x in X2]
X2=[6 if x=='55+' else x for x in X2]

X3=[0 if x=='A' else x for x in X3]
X3=[1 if x=='B' else x for x in X3]
X3=[2 if x=='C' else x for x in X3]

X6=[0 if x=='0' else x for x in X6]
X6=[1 if x=='1' else x for x in X6]
X6=[2 if x=='2' else x for x in X6]
X6=[3 if x=='3' else x for x in X6]
X6=[4 if x=='4+' else x for x in X6]

X[:,0]=np.array(X0).T
X[:,2]=np.array(X2).T
X[:,3]=np.array(X3).T
X[:,6]=np.array(X6).T

X1=X[:300,:]
print(X1)

[[1 10 0 ... 0 3 2]
 [0 16 6 ... 0 1 4]
 [0 15 2 ... 0 1 3]
 ...
 [0 7 2 ... 1 5 1]
 [0 0 2 ... 0 10 1]
 [1 0 1 ... 1 11 3]]


### Derive neighbors matrix A:

In [8]:
def neighbors_matrix(data,theta):
    m,n=np.shape(data)
    A=np.zeros((m,m))
    for i in range(m):
        for j in range(m):
            if jaccard_similarity_score(np.reshape(list(data[i,:]),-1),np.reshape(list(data[j,:]),-1))>=theta:
                A[i,j]=1
    return A.astype(int)

A=neighbors_matrix(X1,0.5)
print(A)

[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]


### Compute the $link(C_i,C_j)$ of $C_i$ and $C_j$

In [30]:
def link(A,c1,c2):
    m,n=np.shape(A)
    sum_=0
    if isinstance(c1,int)==True:
        n1=1
    else:
        n1=len(c1)
    if isinstance(c2,int)==True:
        n2=1
    else:
        n2=len(c2)
    for i in range(n1):
        for j in range(n2):
            if isinstance(c1,int)==True:
                k1=c1
            else:
                k1=c1[i]
            if isinstance(c2,int)==True:
                k2=c2
            else:
                k2=c2[j]
            sum_+=A[k1,k2]
    return sum_

### Compute goodness of $C_i$ and $C_j$

In [10]:
def goodness(A,c1,c2,theta):
    if isinstance(c1,int)==True:
        n1=1
    else:
        n1=len(c1)
    if isinstance(c2,int)==True:
        n2=1
    else:
        n2=len(c2)
    ftheta=(1-theta)/(1+theta)
    if c1==c2:
        gm=0
    else:
        gm=link(A,c1,c2)/((n1+n2)**(1+2*ftheta)-n1**(1+2*ftheta)-n2**(1+2*ftheta))
    return gm

### Find the best pair

In [21]:
def find_best_pair(A,clusters,theta):
    maximum_goodness = 0.0;
    cluster_indexes = [-1, -1];
        
    for i in range(0, len(clusters)):
        for j in range(i + 1, len(clusters)):
            gm = goodness(A,clusters[i], clusters[j],theta);
            if (gm > maximum_goodness):
                maximum_goodness = gm;
                cluster_indexes = [i, j]
        
    return cluster_indexes;

In [24]:
def merge(clusters,cluster_indexes):
    if isinstance(clusters[cluster_indexes[0]],int)==True:
        clusters[cluster_indexes[0]]=[clusters[cluster_indexes[0]]]
    if isinstance(clusters[cluster_indexes[1]],int)==True:
        clusters[cluster_indexes[1]]=[clusters[cluster_indexes[1]]]
    clusters[cluster_indexes[0]].extend(clusters[cluster_indexes[1]])
    clusters.pop(cluster_indexes[1])
    
    return clusters

### Process and clustering result

In [29]:
clusters=list([i for i in range(300)])
for i in range(len(clusters)-10):
    new_clusters=merge(clusters,find_best_pair(A,clusters,0.5))
    if len(new_clusters)<=10:
        print("Number of clusters<=10")
        print(clusters)
        break;
    else:
        clusters=new_clusters

Number of clusters<=10
[[0, 18, 83, 190, 206, 48, 72, 96, 115, 149, 196, 5, 10, 36, 66, 197, 215, 103, 163, 192, 244, 272, 111, 135], [1, 13, 16, 24, 28, 49, 58, 63, 81, 82, 84, 97, 110, 114, 116, 117, 125, 139, 170, 177, 181, 184, 202, 221, 247, 251, 255, 274, 289, 175, 238, 278, 283, 91, 128, 140, 14, 218, 226, 257, 258, 80, 166, 223, 161, 267, 19, 252, 256, 159, 213, 70, 122, 185, 188, 195, 107, 2, 231, 55, 143, 118, 209, 266, 60, 210, 224, 74, 178, 38, 158, 89, 98, 127, 141, 241, 54, 8, 245, 240, 131, 187, 172, 263, 225, 295, 296, 37, 67, 65, 148, 198, 186, 234, 269, 292, 11, 237, 30, 87, 108, 271, 182, 168, 298, 20, 35, 132, 222, 230, 165, 205, 212, 219, 254, 40, 105, 208, 216, 138, 21, 33, 59, 153, 201, 193, 220, 43, 44, 104, 176, 273, 157, 293, 249, 53, 99, 156, 229, 261, 285], [3, 6, 12, 22, 41, 45, 100, 121, 126, 144, 145, 173, 279, 73, 90, 200, 243, 259, 265, 268, 294, 79, 112, 113, 130, 62, 207, 204, 277, 77, 102, 211, 253, 291, 57, 106, 286, 236, 250, 39, 92, 46, 235, 61, 1